In [129]:
import json 
import random
import datetime
import numpy as np

from difflib import SequenceMatcher

def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)
    
def save_json(filename, obj):
    with open(filename, "w+", encoding="utf-8") as f:
        return json.dump(obj, f, ensure_ascii=False, indent=4) 
    
def load_file(filename):
    return list(open(filename, encoding="utf-8").readlines())

In [130]:
# Load current event list

tut = load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\2020_5_8_0\\tutby.json")
cd = load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\2020_5_8_0\\citydog.json")
relax = load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\2020_5_8_0\\relax.json")

current_events = []
current_events.extend(tut)
current_events.extend(cd)
current_events.extend(relax)

In [131]:
old_events = load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\old_events.json")
old_events.extend(load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\relax.json"))
old_events.extend(load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\tutby.json"))
old_events.extend(load_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\citydog.json"))

In [132]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_place_name(event):
    if 'place' not in event:
        return None
    if 'name' not in event['place']:
        return None
    return event['place']['name']

def remove_duplicated(evs):
    removed_idx = set()
    unique_events = []
    for i, current_event in enumerate(evs):
        if i in removed_idx:
            continue
            
        duplicates = [current_event]
        for j, event in enumerate(evs[i + 1:]):
            j += (i + 1)
            if j in removed_idx:
                continue
            res = similar(current_event['title'] + " " + current_event['description'], event['title'] + " " + event['description'])
            if res > 0.55:
                duplicates.append(event)
                removed_idx.add(j)
            elif get_place_name(current_event) == get_place_name(event):
                res = similar(current_event['title'], event['title'])
                if res > 0.7:
                    duplicates.append(event)
                    removed_idx.add(j)
                
        unique_event = max(duplicates, key=lambda e: len(e['title'] + e['description']))
        unique_events.append(unique_event)
        
    return unique_events

def group_by_dates(events):
    events_by_dates = {}
    for event in events:
        date = event['dates'][0]
        if 'start' in date:
            date = date['start']
        if date:
            if "hour" in date:
                dt = datetime.datetime(year=date['year'], month=date['month'], day=date['day'], hour=date['hour'], minute=date['minute'])
            else:
                dt = datetime.date(year=date['year'], month=date['month'], day=date['day'])
        else:
            dt = datetime.date(year=datetime.MINYEAR, month=1, day=1)

        if dt not in events_by_dates:
            events_by_dates[dt] = []
        events_by_dates[dt].append(event)
    return events_by_dates

def remove_duplicated_events(events):
    events_by_dates = group_by_dates(events)
    filtered_events = []
    for dt, grouped_events in events_by_dates.items():
        initial_len = len(grouped_events)
        if initial_len > 1:
            grouped_events = remove_duplicated(grouped_events)
        filtered_events.extend(grouped_events)
    return filtered_events

In [133]:
filtered_events = remove_duplicated_events(current_events)

In [134]:
print("Original size: %d" % len(current_events))
print("Filtered size: %d" % len(filtered_events))

Original size: 862
Filtered size: 717


In [135]:
# Adding old set of events
filtered_events += old_events
print("Original size: %d" % len(filtered_events))
filtered_events = remove_duplicated_events(filtered_events)
print("Filtered size: %d" % len(filtered_events))

Original size: 3928
Filtered size: 2495


In [137]:
# Save results
save_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\2020_5_8_0\\events.json", filtered_events)

# Remove global duplicates (for training)

In [225]:
import urlextract
import re
import math

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter

class TextPreprocessor:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.url_extractor = urlextract.URLExtract()
        self.tag_regex = re.compile(r"<[^>]*>")
        self.email_regex = re.compile(r"[^\s]+@[^\s]+")
        self.number_regex = re.compile(r'\d+(?:\.\d*(?:[eE]\d+))?')
        self.dollar_regex = re.compile(r"[$]+")
        self.spaces_regex = re.compile(r"\s+")
        self.special_chars = [
            "<", "[", "^", ">", "+", "?", "!", "'", ".", ",", ":",
            "*", "%", "#", "_", "=", "-", "&", '/', '\\', '(', ')', ";", "\"", "«", "»", "|", "•", "—", "–", "●", "►", "\n"
        ]
        self.stop_words = set(stopwords.words('russian'))

    def preprocess_text(self, text):
        text = text.lower()
        text = self.remove_html_tags(text)
        text = self.replace_urls(text)
        text = self.replace_emails(text)
        text = self.replace_numbers(text)
        text = self.replace_dollar_signs(text)
        text = self.remove_special_characters(text)
        text = self.remove_stop_words(text)
        text = self.spaces_regex.sub(' ', text)
        return text.strip()

    def remove_html_tags(self, text):
        text = self.tag_regex.sub(" ", text).split(" ")
        text = filter(len, text)
        text = ' '.join(text)
        return text

    def replace_urls(self, text):
        urls = list(set(self.url_extractor.find_urls(text)))
        urls.sort(key=lambda u: len(u), reverse=True)
        for url in urls:
            text = text.replace(url, " httpaddr ")
        return text

    def replace_emails(self, text):
        return self.email_regex.sub(" emailaddr ", text)
    
    def replace_numbers(self, text):
        return self.number_regex.sub(" number ", text)

    def replace_dollar_signs(self, text):
        return self.dollar_regex.sub(" dollar ", text)

    def remove_special_characters(self, text):
        for char in self.special_chars:
            text = text.replace(str(char), " ")
        return text
    
    def remove_stop_words(self, text):
        for word in self.stop_words:
            text = text.replace(" %s " % word, " ")
        return text

preprocessor = TextPreprocessor()
def tokenize(text):
    text = preprocessor.preprocess_text(text)
    return [t.lower() for t in word_tokenize(text)]

def calculate_tf_idf(documents):
    words_freqs = []
    for i, document in enumerate(documents):
        words_freqs.append({})
        token_counts = Counter(tokenize(document))
        total = sum([c for (w, c) in token_counts.items()])
        for token, count in token_counts.items():
            words_freqs[-1][token] = count / total

    words_idf = []
    for i, document in enumerate(documents):
        words_idf.append({})
        freqs = words_freqs[i]
        for token, freq in freqs.items():
            docs_with_token = len([doc for j, doc in enumerate(documents) if token in words_freqs[j]])
            idf = math.log(len(documents) / docs_with_token)
            words_idf[-1][token] = freq * idf
    return words_idf

def get_top(i, k):
    return [w for w, c in sorted(words_idf[i].items(), key=lambda p:[1], reverse=True)[:k]]

In [226]:
documents = [e['title'].strip() + "\n" + e['description'].strip() for e in filtered_events]
words_idf = calculate_tf_idf(documents)

In [232]:
k = 5
idx_to_remove = set()
global_filtered_events = []

for i in range(len(documents)):
    if i in idx_to_remove:
        continue
    duplicates = [i]
    i_set = set(get_top(i, k))
    for j in range(i + 1, len(documents)):
        j_set = set(get_top(j, k))
        rate = len(list(j_set & i_set)) / k
        if rate >= 0.75:
            idx_to_remove.add(j)
            duplicates.append(j)
    unique_event_id = max(duplicates, key=lambda event_id: len(filtered_events[event_id]['tags']))
    global_filtered_events.append(filtered_events[unique_event_id])

In [233]:
len(global_filtered_events)

1975

In [230]:
save_json("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\2020_5_8_0\\events_reduced.json", global_filtered_events)

# Debug

In [ ]:
grouped = group_by_dates(filtered_events)

In [ ]:
may = grouped[datetime.datetime(2020, 10, 23, 19, 0)]

In [ ]:
[e['title'] for e in may]

In [150]:
may[-2]

{'title': 'Stand Up: Павел Воля',
 'short_description': '',
 'description': 'Павел Воля - один из самых дерзких резидентов Comedy Club на ТНТ - везет в Минск новую stand-up программу!\nВы – любитель интеллектуального юмора? Желаете от души повеселиться? Встречайте! Дерзкий и харизматичный комик, мегапопулярный стендапер, острослов и эрудит Павел Воля выступит с новой концертной программой перед зрителями столицы!\xa0\nЗрителей ждет вечер интеллектуального юмора, талантливых импровизаций и невыдуманных жизненных историй!\nЗа 12 лет работы на телеканале ТНТ Павел Воля сумел влюбить в себя многомиллионную аудиторию телезрителей, которая с удовольствием\nего сольные выступления. Сегодня без смелых монологов на самые злободневные темы, приправленных иронией и остроумными шутками, уже невозможно себе представить эту передачу.\nНа сегодняшний день шоумен, телеведущий, актер и музыкант Павел Воля — один из лучших российских комиков, работающих в жанре стенд-ап. Большой stand-up концерт – это я

In [147]:
similar(may[2]['title'] + " " + may[2]['description'], may[-2]['title'] + " " + may[-2]['description'])

0.18588151826888968

In [149]:
similar(may[2]['description'] , may[-2]['description'])

0.18411552346570398

In [122]:
may[2] 

{'title': 'Zoloto / Золото (ПЕРЕНОС)',
 'short_description': '',
 'description': 'ПЕРЕНОС концерта на 17.10.2020 г.\nВремя и место проведения мероприятия остаются прежними.\nБилеты действительны.\nВладимир Золотухин о своем проекте "Творчество в моем случае - это некая психологическая терапия. Мне повезло, возможно, больше других в том плане, что у меня есть какие-то музыкальные навыки и я учусь прислушиваться к себе. У меня очень много вопросов к себе и окружающему меня миру. И творчество - это такая штука где я создаю определенный мир, в котором я могу на эти вопросы ответить. Мои представления о мире я могу выразить музыкой".\nБилет: 25 руб.',
 'poster': 'https://img.afisha.tut.by/static/media/340x0s/cover/01/3/zoloto-zoloto-perenos-321150.jpg',
 'url': 'https://afisha.tut.by/concert/zoloto-zoloto/',
 'place': {'name': 'Корпус',
  'address': 'г. Минск, пр-т Машерова, 9/8',
  'url': 'https://afisha.tut.by/place/korpus-by/'},
 'tags': ['Концерт'],
 'age': None,
 'registration': None,


In [123]:
may[-2]

{'title': 'Zoloto / Золото',
 'short_description': '',
 'description': 'О событии\nВладимир Золотухин о своем проекте "Творчество в моем случае - это некая психологическая терапия. Мне повезло, возможно, больше других в том плане, что у меня есть какие-то музыкальные навыки и я учусь прислушиваться к себе. У меня очень много вопросов к себе и окружающему меня миру. И творчество - это такая штука где я создаю определенный мир, в котором я могу на эти вопросы ответить. Мои представления о мире я могу выразить музыкой".\nБилет: 25 руб.',
 'poster': 'https://img.afisha.tut.by/static/media/340x0s/cover/08/b/zoloto-zoloto-509102.jpg',
 'url': 'https://afisha.tut.by/concert/zoloto-zoloto/',
 'place': {'name': 'Корпус',
  'address': 'г. Минск, пр-т Машерова, 9/8',
  'url': 'https://afisha.tut.by/place/korpus-by/'},
 'tags': ['Концерт'],
 'age': None,
 'registration': None,
 'info': None,
 'cost': None,
 'dates': [{'year': 2020, 'month': 5, 'day': 14, 'hour': 19, 'minute': 0}]}

# Not working

In [152]:
pair = [
    {
        'title': 'Павел Воля Большой Stand-Up',
        'short_description': '',
        'description': 'О событии\nПавел Воля даст концерт в Минске с новой stand-up программой!\nПАВЕЛ ВОЛЯ - один из самых дерзких резидентов Comedy Club на ТНТ - везет в Минск \nновую stand-up программу\n!!!!\nВстречайте! Дерзкий и харизматичный комик, мегапопулярный стендапер, острослов и эрудит Павел Воля выступит с новой концертной программой перед зрителями столицы! \nЗрителей ждет вечер интеллектуального юмора, талантливых импровизаций и невыдуманных жизненных историй!\nНа сегодняшний день шоумен, телеведущий, актер и музыкант Павел Воля — один из лучших российских комиков, работающих в жанре стенд-ап. Его концерты по всему миру неизменно проходят с аншлагами, а билеты на них раскупаются в первые дни продаж.\nНа его выступлениях вы никогда не услышите пошлых и избитых шуток, ведь Павел Воля сам задает тренд в юморе. Павел – первый, кто открывает людям новые шутки и новые тенденции в юморе. То, над чем он пошутил уже сегодня, другие заметят лишь завтра.\nНа концерте\n 28 мая во Дворце Республике в 19:00\n, вы услышите новые монологи на самые важные темы, волнующие современное общество, и, конечно же, шутки, приправленные крепким словцом. \n18 +, Живой звук\nСтоимость билетов: от 60 до 295 рублей\nСправки по телефону: +375 29 696-44-24\n\xa0\nФото и видео',
        'poster': 'https://img.afisha.tut.by/static/media/340x0s/cover/05/7/pavel-volya-bolshoy-stand-up-3224459.jpg',
        'url': 'https://afisha.tut.by/concert/pavel_volya_bolshoy_stand_up/',
        'place': {'name': 'Дворец Республики', 'address': 'г. Минск, пл. Октябрьская, 1', 'url': 'https://afisha.tut.by/place/dvorets-respubliki/'},'tags': ['Концерт', 'Юмор'],
        'age': '18+',
        'registration': None,
        'info': None,
        'cost': None,
        'dates': [{'year': 2020, 'month': 5, 'day': 28, 'hour': 19, 'minute': 0}]
    },
    {
        'title': 'Stand Up: Павел Воля',
        'short_description': '',
        'description': 'Павел Воля - один из самых дерзких резидентов Comedy Club на ТНТ - везет в Минск новую stand-up программу!\nВы – любитель интеллектуального юмора? Желаете от души повеселиться? Встречайте! Дерзкий и харизматичный комик, мегапопулярный стендапер, острослов и эрудит Павел Воля выступит с новой концертной программой перед зрителями столицы!\xa0\nЗрителей ждет вечер интеллектуального юмора, талантливых импровизаций и невыдуманных жизненных историй!\nЗа 12 лет работы на телеканале ТНТ Павел Воля сумел влюбить в себя многомиллионную аудиторию телезрителей, которая с удовольствием\nего сольные выступления. Сегодня без смелых монологов на самые злободневные темы, приправленных иронией и остроумными шутками, уже невозможно себе представить эту передачу.\nНа сегодняшний день шоумен, телеведущий, актер и музыкант Павел Воля — один из лучших российских комиков, работающих в жанре стенд-ап. Большой stand-up концерт – это яркий калейдоскоп, сочетающий импровизации, уморительные шутки, невыдуманные истории из жизни в уникальном и неподражаемом исполнении. Имя Павла Воли уже давно стало синонимом отменного и легкого юмора, актуального для всех возрастных поколений.\nНа концерте\xa028 мая во Дворце Республики\xa0(Октябрьская пл.,1) в 19:00, вы услышите новые монологи на самые важные темы, волнующие современное общество, и, конечно же, шутки, приправленные крепким словцом. Именно вы услышите новую программу!\nНа концертах Павла Воли все счастливы, потому что только счастливый человек может шутить над тем, что его окружает!\nЖивой звук, 18+\n ',
        'poster': 'https://static.relax.by/images/common/afisha/logos/773562894931ff51883a85ba3d6e9681.png',
        'url': 'https://afisha.relax.by/conserts/10512033-koncert-pavla-voli-bolyshoj-stand-up/minsk/',
        'place': {'name': 'Дворец Республики', 'address': 'г. Минск, пл. Октябрьская, 1', 'url': 'https://afisha.relax.by/place/id/6248/'},
        'tags': ['Юмор', 'Концерт'],
        'age': '18+',
        'registration': None,
        'info': None,
        'cost': '64,00 — 299,00 руб.',
        'dates': [{'year': 2020, 'month': 5, 'day': 28, 'hour': 19, 'minute': 0}],
        'raw_dates': ['чт, 28 мая в 19:00']
    }
]

In [ ]:
# Deathstars
# Разное время начала (разница час)

# PLC
# Короткое описание и длинное описание

In [ ]:
pair = [
        {
        "title": "Хелависа",
        "short_description": "",
        "description": "Концерт группы Хелвиса (Мельница) ПЕРЕНОС на 18 ноября. БИЛЕТЫ ДЕЙСТВИТЕЛЬНЫ!\nЛидер группы “Мельница” представит в Минске сольную программу\nНаталья “Хелависа” О`Шей – фронтмен фолк-рок группы “Мельница”, хорошо знакомой большинству беларусов. Композитор, поэт и очаровательной красоты девушка занимается музыкой уже четверть века, в своем творчестве обращаясь к окружающему миру и вдохновляясь стихами классиков. Весной Хелависа приедет в Минск с новой концертной программой.\nСольные концерты Хелависы - нечастое событие. Лидер группы «Мельница» решила отметить десятилетие с выхода своего первого сольного альбома, «Леопард в городе». Этой весной Леопард возвращается в города, путешествует на планету Люцифераза и проходит бесшумными лапами по акустическим хитам «Мельницы». На сцене Хелависе ассистирует, как всегда, Сергей Вишняков.\nклуб «Re:public», 19:00\nБилеты от 45 руб.\nИнфолиния: +375445517706, +375298517706",
        "poster": "https://img.afisha.tut.by/static/media/340x0s/cover/0f/1/khelavisa-5318063-4984176.jpg",
        "url": "https://afisha.tut.by/concert/khelavisa/",
        "place": {
            "name": "Re:Public",
            "address": "г. Минск, ул. Притыцкого, 62",
            "url": "https://afisha.tut.by/place/re-public/"
        },
        "tags": [
            "Концерт"
        ],
        "age": None,
        "registration": None,
        "info": None,
        "cost": None,
        "dates": [
            {
                "year": 2020,
                "month": 11,
                "day": 18,
                "hour": 19,
                "minute": 0
            }
        ],
        "raw_dates": [
            "18 ноября в 19:00"
        ]
    },
    {
        "title": "Хелависа — ПЕРЕНОС",
        "short_description": "",
        "description": "Концерт перенесен на 18.11.2020г.\nК нашему сожалению, заявленный в Минске концерт \"Хелависы\" придётся перенести на более поздний срок. \nНовая дата - 18 ноября \nТакое решение далось непросто и организаторам, и музыкантам. \nЭто не отмена, это — перенос. Причина - форс-мажор, из-за пандемии коронавируса. \nВсе билеты, купленные вами на концерт 12 апреля, будут действительны. \nМы не хотим рисковать вашим здоровьем и жизнями, собирая тысячи человек в закрытом помещении, когда существует хотя бы 1 шанс из 100, что на концерте окажется кто-то из уже заболевших — и потенциально заразный для вас и ваших близких. \nВ это нелёгкое для всех нас время мы вынуждены просить вас о понимании и поддержке. Это очень важно для нас, для артистов и для концертной отрасли в целом. Мы не отменяем мероприятие, концерт пройдет 18 ноября. \nЕсли кто-то из вас решит вернуть билет, по всем вопросам возврата билетов можно будет обратиться после 5 апреля. Тел. +375 29 687-60-68. До этого времени офис организатора концерта закрыт в соответствии с рекомендациями Минздрава Республики Беларусь. \nБез паники, друзья! Берегите себя!\n \n \nНаталья «Хелависа» О`Шей – фронтмен фолк-рок группы «Мельница», хорошо знакомой большинству беларусов. Композитор, поэт и очаровательной красоты девушка занимается музыкой уже четверть века, в своем творчестве обращаясь к окружающему миру и вдохновляясь стихами классиков. Весной Хелависа приедет в Минск с новой концертной программой.\nСольные концерты Хелависы - нечастое событие. Лидер группы «Мельница» решила отметить десятилетие с выхода своего первого сольного альбома, «Леопард в городе». Этой весной Леопард возвращается в города, путешествует на планету Люцифераза и проходит бесшумными лапами по акустическим хитам «Мельницы». На сцене Хелависе ассистирует, как всегда, Сергей Вишняков.\n ",
        "poster": "https://static.relax.by/images/common/afisha/logos/e6553dd05eddcfaf533a1d052ac427a4.png",
        "url": "https://afisha.relax.by/conserts/10929473-helavisa/minsk/",
        "place": {
            "name": "Re:Public",
            "address": "г. Минск, ул. Притыцкого, 62",
            "url": "https://afisha.relax.by/place/id/1039583/"
        },
        "tags": [
            "Рок",
            "Фолк",
            "Концерт"
        ],
        "age": "6+",
        "registration": None,
        "info": None,
        "cost": "48,00 — 68,00 руб.",
        "dates": [
            {
                "year": 2020,
                "month": 11,
                "day": 18,
                "hour": 19,
                "minute": 0
            }
        ],
        "raw_dates": [
            "ср, 18 ноября в 19:00"
        ]
    }
]

In [ ]:
pair = [
        {
        "title": "Мари Краймбрери",
        "short_description": "",
        "description": "О событии\nМари Краймбрери \n— стопроцентный selfmade-артист нового поколения! Несколько лет существуя исключительно в интернет-пространстве, в 2017-ом артистка зазвучала в эфирах федеральных радиостанций и телеканалов и теперь уверенно обосновывается на большой сцене!\nБилеты: 39,00 - 119,00 руб.\nФото и видео",
        "poster": "https://img.afisha.tut.by/static/media/340x0s/cover/10/6/mari-kraymbreri-437848.jpg",
        "url": "https://afisha.tut.by/concert/mari-kraymbreri/",
        "place": {
            "name": "Prime Hall",
            "address": "г. Минск, пр-т Победителей, 65",
            "url": "https://afisha.tut.by/place/prime-hall/"
        },
        "tags": [
            "Концерт",
            "Поп"
        ],
        "age": "16+",
        "registration": None,
        "info": None,
        "cost": None,
        "dates": [
            {
                "year": 2020,
                "month": 12,
                "day": 12,
                "hour": 19,
                "minute": 0
            }
        ]
    },
    {
        "title": "Мари Краймбрери",
        "short_description": "",
        "description": "Большой сольный концерт Мари Краймбрери в Прайм Холл.\nПрезентация нового альбома + все хиты!\nvk.com\n@boulevard_concert\nЛучшая исполнительница 2019 г.\n№1, по мнению подписчиков сообщества «Русская Музыка» \nНа днях TopHit подвёл итоги 2019-го года, благодаря которым:\n- В номинации «Радио взлёт года» Мари Краймбрери поднялась на 61 позицию, заняв второе место в ушедшем году!\n- Работа «AMORE» в номинации «Женский вокал» заняла 4 место, собрав более 450 000 эфиров на радио!\n- Москва, Adrenaline Stadium - #SOLDOUT\n- Санкт-Петербург, A2 Green Concert - #SOLDOUT\n- Победа в номинации «Лучший альбом» («Переобулась») на премии «ЖАРА MUSIC AWARDS 2019»\n- Победа в номинации «Лучшая песня» («На тату») на премии «ЖАРА MUSIC AWARDS 2019»\n- Более 1 млн. прослушиваний треков ВКонтакте.\n- Более 30 млн. просмотров роликов на YouTube.\n ",
        "poster": "https://static.relax.by/images/common/afisha/logos/de0445d8cf3c2acf6169862508124f64.png",
        "url": "https://afisha.relax.by/conserts/10900113-bolyshoj-solynyj-koncert-mari-krajmbreri/minsk/",
        "place": {
            "name": "PRIME HALL",
            "address": "г. Минск, пр-т Победителей, 65",
            "url": "https://afisha.relax.by/place/id/10340894/"
        },
        "tags": [
            "Рэп",
            "Поп",
            "Концерт"
        ],
        "age": None,
        "registration": None,
        "info": None,
        "cost": "39,00 — 119,00 руб.",
        "dates": [
            {
                "year": 2020,
                "month": 12,
                "day": 12,
                "hour": 19,
                "minute": 0
            }
        ],
        "raw_dates": [
            "сб, 12 декабря в 19:00"
        ]
    }
]

In [ ]:
pair = [
    {
        "title": "Мастер-класс «Как влиять на мужчину»",
        "short_description": "",
        "description": "Мастер-класс «Как влиять на мужчину»\nЗанятие в Школе Очарования (для женщин)\nПодтверждение участия по телефону ОБЯЗАТЕЛЬНО. Количество мест ограничено.\nТелефоны для записи +375 29 706 40 07, +375 29 507 97 23\nСсылка на электронную регистрацию:\nhttps://www.delendik.com/wppage/master-class\nСТОИМОСТЬ: 30 р.\t\nИнтерактивное коучинг-занятие от тренера Елены Делендик. Вы узнаете, какие модели поведения и речевые сценарии улучшат общение с вашим мужчиной и внесут гармонию в отношения. Вас также ждут практические упражнения для закрепления теории и приятные бонусы!\nВ ПРОГРАММЕ МЕРОПРИЯТИЯ:\n- Разбор ПОЛЕТОВ в теории, что не так в ваших отношениях?\n- Секрет правильной просьбы: четыре составляющие!\n- Благодарность: простые правила и потенциал на будущее!\n- Как изменить решение мужчины: секрет женского влияния?\n- Как дать совет, когда не просят?\n- Примеры из личной и тренерской практики!\nВстретить свою любовь – это половина дела, еще важнее – понять, как ее сохранить на всю жизнь!\nЗанятие проводит Елена Делендик – руководитель Школы Очарования, сертифицированный коуч, автор обучающих программ и трансформационных игр, тренер с психолого-педагогическим образованием и шестнадцатилетним опытом преподавания, автор книги «Как женщине оставаться женщиной», кандидат наук, предприниматель. О тренере – http://www.delendik.com.\nОсновная цель Школы Очарования – обучать женщин создавать долгосрочные счастливые отношения (раз и навсегда).\nБольшинство наших занятий ориентированы на женщин, которые хотят иметь новый позитивный опыт в отношениях. Результат после обучения – это знание и понимание основ эмоционально здоровых отношений, навыки и умения, необходимые для того, чтобы восстановить доверие, вернуть любовь мужчины и уважение к себе.",
        "poster": "https://img.afisha.tut.by/static/media/340x0s/cover/0f/5/master-klass-kak-vliyat-na-muzhchinu-430949.jpg",
        "url": "https://afisha.tut.by/other/master-klass-kak-vliyat-na-muzhchinu-1/",
        "place": {
            "name": "Бизнес-центр Время",
            "address": "г. Минск, ул. Могилевская, 39А, офис 221",
            "url": "https://afisha.tut.by/place/invastroy/"
        },
        "tags": [
            "",
            "Образовательное"
        ],
        "age": null,
        "registration": null,
        "info": null,
        "cost": null,
        "dates": [
            {
                "year": 2020,
                "month": 6,
                "day": 25,
                "hour": 18,
                "minute": 30
            }
        ],
        "raw_dates": [
            "25 июня в 18:30"
        ]
    },
    {
        "title": "Мастер-класс «Как влиять на мужчину»",
        "short_description": "",
        "description": "Интерактивное коучинг-занятие от тренера Елены Делендик. Вы узнаете, какие модели поведения и речевые сценарии помогут улучшить общение с вашим мужчиной и внесут гармонию в ваши отношения. Вас также ждут практические упражнения для закрепления теории и приятные бонусы!\nВ программе мероприятия:\n– Разбор полетов в теории, что не так в ваших отношениях?\n– Секрет правильной просьбы: четыре составляющие!\n– Благодарность: простые правила и потенциал на будущее!\n– Как изменить решение мужчины: секрет женского влияния?\n– Как дать совет, когда не просят?\n– Примеры из личной и тренерской практики!\nВстретить свою любовь — это половина дела, еще важнее — понять, как ее сохранить на всю жизнь!\nЗанятие проводит Елена Делендик – руководитель Школы Очарования, сертифицированный коуч, автор обучающих программ и трансформационных игр, тренер с психолого-педагогическим образованием и шестнадцатилетним опытом преподавания, автор книги «Как женщине оставаться женщиной», кандидат наук, предприниматель.\nБольшинство наших занятий ориентированы на женщин, которые хотят иметь новый позитивный опыт в отношениях. Результат после обучения – это знание и понимание основ эмоционально здоровых отношений, навыки и умения, необходимые для того, чтобы восстановить доверие, вернуть любовь мужчины и уважение к себе.\nПодтверждение участия по телефону обязательно. Количество мест ограничено.\nТелефоны для записи +375 29 706-40-07, +375 29 507-97-23\nСсылка на электронную регистрацию: \nwww.delendik.com\n ",
        "poster": "https://static.relax.by/images/common/afisha/logos/fa6975fb9e36458a8a55bc51bf25d519.jpg",
        "url": "https://afisha.relax.by/education/10844663-subbotnij-vecher-kak-vlijaty-na-muzhchinu/minsk/",
        "place": {
            "name": "Время",
            "address": "г. Минск, ул. Могилевская, 39а",
            "url": "https://afisha.relax.by/place/id/10640713/"
        },
        "tags": [
            "Мастер-класс",
            "Образовательное"
        ],
        "age": null,
        "registration": "+375 29 706 40 07 +375 29 507 97 23",
        "info": null,
        "cost": "30,00 руб.",
        "dates": [
            {
                "year": 2020,
                "month": 6,
                "day": 25,
                "hour": 18,
                "minute": 30
            }
        ],
        "raw_dates": [
            "чт, 25 июня в 18:30"
        ]
    }
]

In [ ]:
pair = [
     {
        "title": "MGC: Live 2 Fight",
        "short_description": "",
        "description": "Экотехнопарк представляет:\n29 февраля. День силы. День духа. День жестких поединков и крутых парней.\nТакое случается реже, чем раз в 4 года.\nТакого ещё не было в Минске.\nКикбоксинг!\nММА!\nПрофессиональный бокс!\nЧетверка по правилам прайда!\n4 вида единоборств в один день, на одной Арене.\nВ Фэлкон Клаб покажут себя лучшие бойцы Беларуси, Польши, Армении и Азербайджана\nБой вечера: Максим Сподаренко (Беларусь) против Рамала Асланова (Азербайджан)   \nИнтрига вечера - бой-реванш: Жора Акопян (Армения) против Рафаля Дудека (Польша)\nВрубаем адреналин на максимум!  Идем на сильное шоу!\n \nP.S.\nАтмосферу создадут мощные спецэффекты, силовое шоу, жаркий битбокс, изысканные балерины, качающий DJ.\nВсё решится 29.02.20.    \nТурнир будет транслироваться в прямом эфире на стриминговой платформе «FITE».\n ",
        "poster": "https://static.relax.by/images/common/afisha/logos/b4b7f3f05f0a1481adce1d4c572b57e0.png",
        "url": "https://afisha.relax.by/sport/10928343-mgc-live-2-fight/minsk/",
        "place": {
            "name": "Falcon Club Arena",
            "address": "г. Минск, пр-т Победителей, 20",
            "url": "https://afisha.relax.by/place/id/10602525/"
        },
        "tags": [
            "Спорт"
        ],
        "age": null,
        "registration": null,
        "info": null,
        "cost": "45,00 — 200,00 руб.",
        "dates": [
            {
                "year": 2020,
                "month": 2,
                "day": 29,
                "hour": 19,
                "minute": 0
            }
        ]
    },
     {
        "title": "Бои.MGC: Live 2 Fight",
        "short_description": "",
        "description": "О событии\nПромоушен MGC представляет вечер боев 29 февраля в Falcon Club Arena\nМолодой, но очень амбициозный промоушен MGC представляет вашему вниманию турнир с участием сильнейших бойцов Беларуси и Европы.\nЗдесь не будет поединков спортсменов-велосипедистов и вам не захочется уйти уже после первого раунда. Только элита европейской и белорусской школ единоборств. Старый добрый классический бокс с участием опытных братьев Евгения и Михаила Долголевцов, для которых данный турнир станет дебютом в профессионалах и бесценным опытом на пути к олимпийской медали.\nПротивостояние одних из сильнейших \"кикеров\" современности. Лучшего кикбоксера Европы 2018 года по версии WAKO белоруса Максима Сподаренко (73 кг) и чемпиона мира-2019 WAKO, обладателя пяти профессиональных поясов азербайджанца Рамала Асланова. Схватка армянского тигра в весовой категории 73 кг Жоры Акопяна и поляка Рафаэля Дудека. В этом поединке Жора Акопян попробует взять реванш за единственное поражение в профессиональном рейтинге над опытным \"файтером\" Рафаэлем Дудеком, который долго избегал встречи в ринге с Жорой и не хотел проводить реванш.\nВ смешанных единоборствах состоится профессиональный дебют в ММА мастера спорта международного класса по греко-римской борьбе, серебряного призера мирового форума Марселя Сычева. Спортсмен попытается доказать превосходство своего стиля и уже заряжен на эффектную викторию.\nТакже нас ожидает и женское противостояния лучших представительниц своих стран по правилам К1 Александры Ситниковой (Беларусь) и Камиллы Баланды (Польша).\nИ долгожданный бой лидеров сборной Беларуси - серебряного призера чемпионата мира 2019 года Павла Шелеста против серебряного призера мирового форума-2017 Евгения Мягкова.\nУкрашением турнира станет четверка боев по правилам Pridе, где сильнейшие мастера разных видов единоборств в тяжелом весе попробуют доказать превосходство своего вида спорта. В 90-х годах такие поединки производили фурор, а позднее канули в лета. Но организаторы, взяв от них все лучшее, представляют бои в обновленном формате.",
        "poster": "https://img.afisha.tut.by/static/media/340x0s/cover/03/7/boi-mgc-live-2-fight-823183-691429.jpg",
        "url": "https://afisha.tut.by/other/boi-mgc-live-2-fight/",
        "place": {
            "name": "Фалкон Клуб",
            "address": "г. Минск, пр-т Победителей, 20",
            "url": "https://afisha.tut.by/place/falcon-club/"
        },
        "tags": [
            "Спорт"
        ],
        "age": null,
        "registration": null,
        "info": null,
        "cost": null,
        "dates": [
            {
                "year": 2020,
                "month": 2,
                "day": 29,
                "hour": 19,
                "minute": 0
            }
        ]
    }
]